In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
     #   print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf

In [3]:
vgg=tf.keras.applications.VGG16(include_top=False,input_shape=(224,224,3))

58892288/58889256 [==============================] - 0s 0us/step


In [8]:
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Model

In [10]:
for layer in vgg.layers:
    layer.trainable=False

In [11]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
flatten_layer=Flatten()(vgg.output)
final_layer=Dense(2,activation='softmax',)(flatten_layer)

In [17]:
model=Model(inputs=vgg.input,outputs=final_layer)

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [19]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
data_gen=ImageDataGenerator(shear_range=0.2,zoom_range=0.2,rotation_range=0.40,horizontal_flip=0.2,rescale=1/255)

In [22]:
train_data=data_gen.flow_from_directory('../input/face-mask-detection/dataset',target_size=(224,224),batch_size=32,)

Found 440 images belonging to 2 classes.


In [27]:
train_data.class_indices

{'with_mask': 0, 'without_mask': 1}

In [28]:
len(train_data)

14

In [29]:
model.fit_generator(train_data,steps_per_epoch=len(train_data),epochs=7)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/7
14/14 [==============================] - 28s 1s/step - loss: 1.3706 - accuracy: 0.5781
Epoch 2/7
14/14 [==============================] - 12s 832ms/step - loss: 0.1908 - accuracy: 0.9323
Epoch 3/7
14/14 [==============================] - 11s 782ms/step - loss: 0.0673 - accuracy: 0.9749
Epoch 4/7
14/14 [==============================] - 11s 819ms/step - loss: 0.0559 - accuracy: 0.9849
Epoch 5/7
14/14 [==============================] - 11s 801ms/step - loss: 0.0343 - accuracy: 0.9894
Epoch 6/7
14/14 [==============================] - 11s 821ms/step - loss: 0.0385 - accuracy: 0.9881
Epoch 7/7
14/14 [==============================] - 12s 842ms/step - loss: 0.0220 - accuracy: 0.9931


In [30]:
model.save('VGG16_mask_unmask.h5')

In [32]:
new_model=tf.keras.models.load_model('./VGG16_mask_unmask.h5')

In [31]:
import cv2

In [34]:
def get_mask_or_without_mask(filepath):
    img=cv2.imread(filepath)
    img=cv2.resize(img,(224,224))/255
    class_code=new_model.predict(img.reshape(1,224,224,3)).argmax()
    return 'with_mask' if class_code==0 else 'without_mask'

In [35]:
get_mask_or_without_mask('../input/mask-unmask/1618888073046.jpg')

'without_mask'

In [36]:
get_mask_or_without_mask('../input/mask-unmask/1618888073072.jpg')

'with_mask'

In [37]:
get_mask_or_without_mask('../input/mask-unmask/1618888073109.jpg')

'with_mask'

In [38]:
get_mask_or_without_mask('../input/mask-unmask/sm.jpg')

'without_mask'